In [ ]:
!pip install -r requirements.txt

In [ ]:
#!pip install  pdfplumber tqdm


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\kobal\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os, re, json
import pdfplumber
from tqdm import tqdm

from pathlib import Path
import os

BASE_DIR = Path.cwd()          
PDF_DIR  = BASE_DIR / "data" 
OUT_DIR  = BASE_DIR / "parsed"

OUT_DIR.mkdir(parents=True, exist_ok=True)


import re

def clean_text(text):
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = text.replace("\u00ad", "")
    text = re.sub(r"[\u200b\u200c\u200d\ufeff]", "", text)

    
    text = text.replace("", "- ").replace("", "- ").replace("•", "- ")
   
    text = re.sub(r"(?m)^\s*o\s+", "- ", text)

  
    text = re.sub(r"(?m)^\s*=+\s*$", "", text)

   
    text = re.sub(r"\bC\s+opyright\b", "Copyright", text, flags=re.IGNORECASE)

    
    text = re.sub(r"[ \t]+", " ", text)

 
    text = re.sub(r"(?m)^\-\s*$", "", text)
    text = re.sub(r"(?m)^\-\s+[А-ЯA-Z0-9]\s*$", "", text)   

    text = re.sub(r"\n{3,}", "\n\n", text)

    return text.strip()



def parse_pdf(path):
    pages = []
    with pdfplumber.open(path) as pdf:
        for i, page in enumerate(pdf.pages, 1):
            text = page.extract_text() or ""
            text = clean_text(text)
            if len(text) > 200:
                pages.append({
                    "page": i,
                    "text": text
                })
    return pages

def chunk_text(text, max_chars=2000):
    paras = [p.strip() for p in re.split(r"\n\s*\n", text) if p.strip()]
    chunks, cur = [], ""
    for p in paras:
        if len(cur) + len(p) + 2 > max_chars and cur.strip():
            chunks.append(cur.strip())
            cur = p
        else:
            cur = (cur + "\n\n" + p).strip() if cur else p
    if cur.strip():
        chunks.append(cur.strip())
    return chunks



In [ ]:
for fn in tqdm(os.listdir(PDF_DIR)):
    if not fn.lower().endswith(".pdf"):
        continue

    pdf_path = os.path.join(PDF_DIR, fn)
    doc_id = fn.replace(".pdf", "").replace(" ", "_")

    pages = parse_pdf(pdf_path)
    out_path = os.path.join(OUT_DIR, f"{doc_id}.jsonl")

    with open(out_path, "w", encoding="utf-8") as f:
        for page in pages:
            chunks = chunk_text(page["text"])
            for i, ch in enumerate(chunks):
                record = {
                    "doc_id": doc_id,
                    "page": page["page"],
                    "chunk_id": f"{doc_id}_p{page['page']}_c{i}",
                    "text": ch
                }
                f.write(json.dumps(record, ensure_ascii=False) + "\n")


  0%|          | 0/6 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [00:21<01:47, 21.59s/it]

In [ ]:
import itertools

fn = "06_ViPNet_Coordinator_HW_5._История_версий.jsonl"
path = os.path.join(OUT_DIR, fn)

with open(path, encoding="utf-8") as f:
    line = next(itertools.islice(f, 0, 1))
    rec = json.loads(line)
    print(rec["text"])

ViPNet Coordinator HW 5
История версий
Версия продукта 5.3.2
Copyright (c) InfoTeCS. All Rights Reserved
Что нового в версии 5.3.1
Изменения версии 5.3.1 по сравнению с версией 5.3.0.
- Прекращена поддержка автоматического определения видимости узлов и сетей ViPNet
Для повышения стабильности работы сети ViPNet больше не поддерживается автоматическое
определение видимости узлов и сетей ViPNet. При обновлении такая видимость будет
преобразована в видимость по реальным или виртуальным IP-адресам.
- Изменен веб-интерфейс подсчета срабатываний сетевых фильтров и правил трансляции
адресов
Теперь для выбора режима подсчета срабатываний используется переключатель
Временный подсчет.
- Заменены термины документации
- кластер горячего резервирования –> кластер;
- журнал регистрации IP-пакетов –> журнал IP-пакетов.
Что нового в версии 5.3.0
Изменения и новые возможности версии 5.3.0 по сравнению с версией 5.2.1.
- Поддержка протокола BGP
Добавлена поддержка протокола BGP, который используется для 

In [ ]:
import glob

stats = []

for path in OUT_DIR.glob("*.jsonl"):
    n = 0
    total_len = 0

    with open(path, encoding="utf-8") as f:
        for line in f:
            rec = json.loads(line)
            n += 1
            total_len += len(rec["text"])

    stats.append((path.name, n, total_len // max(n, 1)))

sorted(stats, key=lambda x: -x[1])[:10]



[('04_ViPNet_Coordinator_HW_5._Справочник.jsonl', 537, 1430),
 ('02_ViPNet_Coordinator_HW_5._Настройка_в_CLI.jsonl', 472, 1645),
 ('03_ViPNet_Coordinator_HW_5._Настройка_в_WEB.jsonl', 336, 1370),
 ('01_ViPNet_Coordinator_HW_5._Подготовка_к_работе.jsonl', 91, 1264),
 ('06_ViPNet_Coordinator_HW_5._История_версий.jsonl', 10, 2229),
 ('07_ViPNet_Coordinator_HW_5._Перечень_совместимых_трансиверов.jsonl',
  8,
  920)]

In [ ]:
#!pip install  sentence-transformers faiss-cpu


   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/18.9 MB 1.7 MB/s eta 0:00:12
   ---------------------------------------- 0.1/18.9 MB 787.7 kB/s eta 0:00:24
   ---------------------------------------- 0.1/18.9 MB 744.7 kB/s eta 0:00:26
   ---------------------------------------- 0.1/18.9 MB 599.1 kB/s eta 0:00:32
   ---------------------------------------- 0.1/18.9 MB 599.1 kB/s eta 0:00:32
   ---------------------------------------- 0.2/18.9 MB 538.9 kB/s eta 0:00:35
   ---------------------------------------- 0.2/18.9 MB 551.6 kB/s eta 0:00:34
   ---------------------------------------- 0.2/18.9 MB 562.0 kB/s eta 0:00:34
   ---------------------------------------- 0.2/18.9 MB 518.8 kB/s eta 0:00:36
   --------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\kobal\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np

PARSED_DIR = OUT_DIR



records = []
for path in PARSED_DIR.glob("*.jsonl"):
    with open(path, encoding="utf-8") as f:
        for line in f:
            r = json.loads(line)
            txt = r["text"].strip()
            if len(txt) < 200:
                continue
            records.append({
                "doc_id": r["doc_id"],
                "chunk_id": r["chunk_id"],
                "page": r.get("page"),
                "text": txt
            })

len(records), records[0].keys()



(1454, dict_keys(['doc_id', 'chunk_id', 'page', 'text']))

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

model = SentenceTransformer("intfloat/multilingual-e5-base")

def embed_texts(texts, batch_size=64):
    embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        
        batch = ["passage: " + t for t in batch]
        e = model.encode(batch, normalize_embeddings=True, show_progress_bar=False)
        embs.append(e)
    return np.vstack(embs).astype("float32")

texts = [r["text"] for r in records]
emb = embed_texts(texts)

index = faiss.IndexFlatIP(emb.shape[1])  
index.add(emb)

index.ntotal


C:\Users\kobal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\kobal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kobal\.cache\huggingface\hub\models--intfloat--multilingual-e5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/hu

1454

In [ ]:
def search(query, k):
    q = model.encode(["query: " + query], normalize_embeddings=True).astype("float32")
    scores, ids = index.search(q, k)
    out = []
    for score, idx in zip(scores[0], ids[0]):
        r = records[int(idx)]
        out.append((float(score), r))
    return out

def show_results(query, k=5, max_chars=600):
    res = search(query, k=k)
    print("QUERY:", query)
    print("="*80)
    for i, (score, r) in enumerate(res, 1):
        print(f"[{i}] score={score:.3f} doc={r['doc_id']} page={r.get('page')} chunk={r['chunk_id']}")
        print(r["text"][:max_chars])
        print("-"*80)

In [ ]:
show_results("Как подключиться к веб-интерфейсу ViPNet Coordinator HW?", k=5)
show_results("Какая команда показывает таблицу маршрутизации?", k=5)
show_results("Как настроить BGP?", k=5)


QUERY: Как подключиться к веб-интерфейсу ViPNet Coordinator HW?
[1] score=0.910 doc=03_ViPNet_Coordinator_HW_5._Настройка_в_WEB page=18 chunk=03_ViPNet_Coordinator_HW_5._Настройка_в_WEB_p18_c0
- Чтобы добавить IP-адреса сетевых интерфейсов и доменные имена, по которым вы будете
подключаться.
Вместо самоподписанного сертификата вы можете использовать сертификат, выданный УЦ.
Подробнее о том, как перевыпустить самоподписанный сертификат и использовать сертификат УЦ
см. в разделе Настройка подключения с помощью веб-интерфейса.
Если на ViPNet Coordinator HW запущен прокси-сервер в обычном режиме, то в настройках
параметров локальной сети компьютера:
1 Укажите IP-адрес и порт прокси-сервера.
2 Исключите использование прокси-сервера для локальных адресов, в том числе адресов ViPNet
Coordi
--------------------------------------------------------------------------------
[2] score=0.908 doc=03_ViPNet_Coordinator_HW_5._Настройка_в_WEB page=15 chunk=03_ViPNet_Coordinator_HW_5._Настройка_в_WEB_p15

In [ ]:
from sentence_transformers import CrossEncoder

reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def search_rerank(query, k=5, pre_k=20):
    q = model.encode(["query: " + query], normalize_embeddings=True).astype("float32")
    scores, ids = index.search(q, pre_k)

    candidates = [records[int(i)] for i in ids[0]]
    pairs = [(query, c["text"]) for c in candidates]

    
    rr_scores = reranker.predict(pairs)
    ranked = sorted(
        zip(rr_scores, candidates),
        key=lambda x: -x[0]
    )

    return ranked[:k]

def show_rerank(query, k=5):
    res = search_rerank(query, k=k)
    print("QUERY:", query)
    print("="*80)
    for i, (score, r) in enumerate(res, 1):
        print(f"[{i}] rerank_score={score:.3f} doc={r['doc_id']} page={r.get('page')}")
        print(r["text"][:600])
        print("-"*80)


In [ ]:
benchmark = [
    {
        "question": "Как подключиться к веб-интерфейсу ViPNet Coordinator HW?",
        "doc_id": "03_ViPNet_Coordinator_HW_5._Настройка_в_WEB"
    },
    {
        "question": "Какая команда показывает таблицу маршрутизации?",
        "doc_id": "02_ViPNet_Coordinator_HW_5._Настройка_в_CLI"
    },
    {
        "question": "Как настроить BGP?",
        "doc_id": "03_ViPNet_Coordinator_HW_5._Настройка_в_WEB"
    },
    {
        "question": "Как добавить соседа BGP?",
        "doc_id": "03_ViPNet_Coordinator_HW_5._Настройка_в_WEB"
    },
    {
        "question": "Как настроить анонсирование подсетей в BGP?",
        "doc_id": "03_ViPNet_Coordinator_HW_5._Настройка_в_WEB"
    },
]


In [ ]:
def recall_at_k(question, true_doc_id, k=10):
    res = search(question, k=k)
    docs = [r["doc_id"] for _, r in res]
    return int(true_doc_id in docs)

scores = []
for q in benchmark:
    scores.append(recall_at_k(q["question"], q["doc_id"], k=5))

scores


[1, 1, 1, 1, 1]

In [ ]:
k = 5
pre_k = 30

recall_base = []
precision_base = []

recall_rr = []
precision_rr = []

for q in benchmark:
    question = q["question"]
    true_doc = q["doc_id"]

    
    base = search(question, k=k)
    base_docs = [r["doc_id"] for _, r in base]
    recall_base.append(int(true_doc in base_docs))
    precision_base.append(sum(d == true_doc for d in base_docs) / k)

    
    rr = search_rerank(question, k=k, pre_k=pre_k)
    rr_docs = [r["doc_id"] for _, r in rr]
    recall_rr.append(int(true_doc in rr_docs))
    precision_rr.append(sum(d == true_doc for d in rr_docs) / k)

print("Baseline:")
print("  Recall@{}   = {:.2f}".format(k, sum(recall_base) / len(recall_base)))
print("  Precision@{}= {:.2f}".format(k, sum(precision_base) / len(precision_base)))

print("\nRerank:")
print("  Recall@{}   = {:.2f}".format(k, sum(recall_rr) / len(recall_rr)))
print("  Precision@{}= {:.2f}".format(k, sum(precision_rr) / len(precision_rr)))



Baseline:
  Recall@5   = 1.00
  Precision@5= 0.52

Rerank:
  Recall@5   = 1.00
  Precision@5= 0.44


In [ ]:
import os, glob, json, re
import numpy as np

PARSED_DIR = OUT_DIR
DOC_PREFIX = "02_ViPNet_Coordinator_HW_5._Настройка_в_CLI"  

doc_path = os.path.join(PARSED_DIR, DOC_PREFIX + ".jsonl")
assert os.path.exists(doc_path), doc_path

def looks_like_toc_line(line: str) -> bool:
    return line.count(".") >= 10 or "...." in line

chunks = []
with open(doc_path, encoding="utf-8") as f:
    for line in f:
        r = json.loads(line)
        chunks.append({
            "chunk_id": r["chunk_id"],
            "page": int(r.get("page", 0)),
            "text": r["text"]
        })

len(chunks), chunks[0]["page"], chunks[0]["chunk_id"]

CHAPTER_RE = re.compile(r"^(Глава\s+\d+\.?.+)$", re.IGNORECASE)
SECTION_RE = re.compile(r"^(\d+\.\d+(\.\d+)*\s+.+)$") 

def extract_headings_from_text(text):
    headings = []
    for raw in text.split("\n"):
        line = raw.strip()
        if not line:
            continue
        if looks_like_toc_line(line):
            continue

        m = CHAPTER_RE.match(line)
        if m:
            headings.append(("chapter", m.group(1)))
            continue

        m = SECTION_RE.match(line)
        if m and len(line) <= 120:
            headings.append(("section", m.group(1)))
    return headings

In [ ]:
chunks_sorted = sorted(chunks, key=lambda x: (x["page"], x["chunk_id"]))

tree = []  
cur_ch = None
cur_sec = None

def ensure_chapter(title, page):
    return {"title": title, "start_page": page, "end_page": page, "sections": []}

def ensure_section(title, page):
    return {"title": title, "start_page": page, "end_page": page}

for c in chunks_sorted:
    page = c["page"]
    hs = extract_headings_from_text(c["text"])

    if cur_sec:
        cur_sec["end_page"] = page
    if cur_ch:
        cur_ch["end_page"] = page

    for kind, title in hs:
        if kind == "chapter":
            if cur_ch:
                tree.append(cur_ch)
            cur_ch = ensure_chapter(title, page)
            cur_sec = None
        elif kind == "section":
            if cur_ch is None:
        
                cur_ch = ensure_chapter("CHAPTER_UNKNOWN", page)
            cur_sec = ensure_section(title, page)
            cur_ch["sections"].append(cur_sec)


if cur_ch:
    tree.append(cur_ch)

len(tree), tree[0]["title"], len(tree[0]["sections"])

TREE_PATH = OUT_DIR / f"{DOC_PREFIX}_tree_index.json"
TREE_PATH.write_text(
    json.dumps(tree, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

TREE_PATH

(1, 'CHAPTER_UNKNOWN', 254)

In [ ]:
model = SentenceTransformer("intfloat/multilingual-e5-base")

def embed(texts):
    texts = ["passage: " + t for t in texts]
    return model.encode(texts, normalize_embeddings=True).astype("float32")


section_nodes = []
for ch in tree:
    for sec in ch["sections"]:
        section_nodes.append({
            "chapter": ch["title"],
            "section": sec["title"],
            "start_page": sec["start_page"],
            "end_page": sec["end_page"],
            "title": f"{ch['title']} | {sec['title']}"
        })

section_titles = [n["title"] for n in section_nodes]
sec_emb = embed(section_titles)

sec_index = faiss.IndexFlatIP(sec_emb.shape[1])
sec_index.add(sec_emb)

len(section_nodes)

254

In [ ]:
def pageindex_like_search(query, k_sections=3, k_chunks=5):
    q = model.encode(["query: " + query], normalize_embeddings=True).astype("float32")

    
    sec_scores, sec_ids = sec_index.search(q, k_sections)
    picked = [section_nodes[int(i)] for i in sec_ids[0]]

    
    candidate_chunks = []
    for node in picked:
        for c in chunks_sorted:
            if node["start_page"] <= c["page"] <= node["end_page"]:
                candidate_chunks.append(c)

    
    if len(candidate_chunks) < 50:
        candidate_chunks = chunks_sorted

    cand_texts = [c["text"] for c in candidate_chunks]
    cand_emb = embed(cand_texts)

    tmp_index = faiss.IndexFlatIP(cand_emb.shape[1])
    tmp_index.add(cand_emb)

    scores, ids = tmp_index.search(q, k_chunks)
    out = []
    for score, idx in zip(scores[0], ids[0]):
        out.append((float(score), candidate_chunks[int(idx)], picked))
    return out


def show_pageindex_like(query, k_sections=3, k_chunks=5, max_chars=500):
    res = pageindex_like_search(query, k_sections=k_sections, k_chunks=k_chunks)
    print("QUERY:", query)
    print("="*80)
    
    picked = res[0][2]
    print("Picked sections (Stage A):")
    for n in picked:
        print(f"- {n['title']}  pages {n['start_page']}-{n['end_page']}")
    print("-"*80)

    for i, (score, c, _) in enumerate(res, 1):
        print(f"[{i}] score={score:.3f} page={c['page']} chunk={c['chunk_id']}")
        print(c["text"][:max_chars])
        print("-"*80)


In [ ]:
show_pageindex_like("Какая команда показывает таблицу маршрутизации?", k_sections=3, k_chunks=5)
show_pageindex_like("Как настроить BGP?", k_sections=3, k_chunks=5)

QUERY: Какая команда показывает таблицу маршрутизации?
Picked sections (Stage A):
- CHAPTER_UNKNOWN | 2.2 Создайте таблицу PROXY-DEFAULT для маршрутизации трафика до прокси-сервера через  pages 249-249
- CHAPTER_UNKNOWN | 3.2 Настройте маршрутизацию к устройству Static NAT:  pages 57-57
- CHAPTER_UNKNOWN | 2.5 Активируйте политику маршрутизации:  pages 249-278
--------------------------------------------------------------------------------
[1] score=0.855 page=202 chunk=02_ViPNet_Coordinator_HW_5._Настройка_в_CLI_p202_c0
Атрибут Расшифровка в командном Пояснение
интерпретаторе
> selected route Лучший маршрут — отображается, если в таблице есть несколько
маршрутов в одну и ту же сеть с разной административной
дистанцией.
* FIB route Маршрут, признанный лучшим и участвующий в маршрутизации.
Примечание. В таблице маршрутизации могут присутствовать другие атрибуты, кроме описанных в
таблице выше. Эти атрибуты зарезервированы для других протоколов маршрутизации и в текущей
версии ViPNet Coo

In [ ]:
WINDOW = 10

max_page = max(c["page"] for c in chunks_sorted)
page_nodes = []
for start in range(1, max_page+1, WINDOW):
    end = min(max_page, start + WINDOW - 1)
    title = f"Pages {start}-{end}"
    page_nodes.append({"title": title, "start_page": start, "end_page": end})


def window_text(start, end, limit_chars=4000):
    parts = []
    for c in chunks_sorted:
        if start <= c["page"] <= end:
            parts.append(c["text"])
            if sum(len(p) for p in parts) > limit_chars:
                break
    return "\n".join(parts)[:limit_chars]

window_texts = [window_text(n["start_page"], n["end_page"]) for n in page_nodes]
win_emb = embed(window_texts)

win_index = faiss.IndexFlatIP(win_emb.shape[1])
win_index.add(win_emb)
len(page_nodes)


def pagewindow_search(query, k_windows=3, k_chunks=5):
    q = model.encode(["query: " + query], normalize_embeddings=True).astype("float32")

    win_scores, win_ids = win_index.search(q, k_windows)
    picked = [page_nodes[int(i)] for i in win_ids[0]]

    candidate_chunks = []
    for w in picked:
        for c in chunks_sorted:
            if w["start_page"] <= c["page"] <= w["end_page"]:
                candidate_chunks.append(c)

    cand_texts = [c["text"] for c in candidate_chunks]
    cand_emb = embed(cand_texts)

    tmp = faiss.IndexFlatIP(cand_emb.shape[1])
    tmp.add(cand_emb)

    scores, ids = tmp.search(q, k_chunks)
    out = []
    for score, idx in zip(scores[0], ids[0]):
        out.append((float(score), candidate_chunks[int(idx)], picked))
    return out

def show_pagewindow(query, k_windows=3, k_chunks=5, max_chars=500):
    res = pagewindow_search(query, k_windows=k_windows, k_chunks=k_chunks)
    print("QUERY:", query)
    print("="*80)
    print("Picked windows (Stage A):")
    for w in res[0][2]:
        print(f"- {w['title']}")
    print("-"*80)
    for i, (score, c, _) in enumerate(res, 1):
        print(f"[{i}] score={score:.3f} page={c['page']} chunk={c['chunk_id']}")
        print(c["text"][:max_chars])
        print("-"*80)
show_pagewindow("Какая команда показывает таблицу маршрутизации?", k_windows=10, k_chunks=5)
show_pagewindow("Как настроить BGP?", k_windows=10, k_chunks=5)


QUERY: Какая команда показывает таблицу маршрутизации?
Picked windows (Stage A):
- Pages 201-210
- Pages 241-250
- Pages 221-230
- Pages 211-220
- Pages 21-30
- Pages 391-400
- Pages 31-40
- Pages 231-240
- Pages 401-410
- Pages 101-110
--------------------------------------------------------------------------------
[1] score=0.855 page=202 chunk=02_ViPNet_Coordinator_HW_5._Настройка_в_CLI_p202_c0
Атрибут Расшифровка в командном Пояснение
интерпретаторе
> selected route Лучший маршрут — отображается, если в таблице есть несколько
маршрутов в одну и ту же сеть с разной административной
дистанцией.
* FIB route Маршрут, признанный лучшим и участвующий в маршрутизации.
Примечание. В таблице маршрутизации могут присутствовать другие атрибуты, кроме описанных в
таблице выше. Эти атрибуты зарезервированы для других протоколов маршрутизации и в текущей
версии ViPNet Coordinator HW не используются
--------------------------------------------------------------------------------
[2] score=0.846 p